In [59]:
from two_step_classification import *
print_version()

ran:  2023-09-20 13:05:43.739514
python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [60]:
results: list[dict[str, ]] = []

In [61]:
# load data
data_paths = ["models/kdd99_features/x_test_df.pkl",
              "models/kdd99_features/x_test-drop_25_df.pkl",
              "models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test-drop+ae_40_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test+ae_43_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test+ae_48_df&activation=relu&epochs=5&batch_size=32.pkl",
              ]
lgb_binary_paths = ["models/lightgbm/lgb_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped_25_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_30_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_35_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_40_binary_tuned_booster.model",
                    "models/lightgbm/lgb+ae_43_binary_tuned_booster.model",
                    "models/lightgbm/lgb+ae_48_binary_tuned_booster.model",
                    ]
lgb_anomaly_paths = ["models/lightgbm/lgb_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped_25_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_30_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_35_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_40_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb+ae_43_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb+ae_48_mapped_anomaly_tuned_booster.model",]



In [93]:
# y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
# y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)
# y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")

def predict(x_path: str, path_1st: str, path_2nd: str):
    """
    2段階分類の予測を行う
    Args:
        x_path: 検証データのパス
        path_1st: 第1段階のモデルのパス
        path_2nd: 第二段階のモデルのパス

    Returns: 
        

    """
    y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
    y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)
    # y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")
    x_test = pd.read_pickle(x_path)
    model_1st: lgb.Booster = lgb.Booster(model_file=path_1st)
    model_2nd: lgb.Booster = lgb.Booster(model_file=path_2nd)
    
    # 第1段階: 正常と異常の2値分類
    y_pred_binary:pd.Series = classification_normal_and_anomaly(x_test, model_1st)
    predicted_indexes = y_pred_binary[y_pred_binary == 1].index
    
    y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
    
    cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])
    
    # 第二段階；異常の分類
    x_anomalies: pd.DataFrame = x_test.loc[predicted_indexes]
    y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
    y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
    # print(f"{y_pred_anomalies.value_counts()}")
    
    y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
    cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())
    
    swapped_correspondences = {v: k for k, v in correspondences.items()}
    y_pred_value_counts = y_pred.value_counts()
    y_pred_value_counts.index = y_pred_value_counts.index.map(lambda x: swapped_correspondences[x])
    # print(y_pred_value_counts)
    
    # y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
    # predicted_indexes = y_pred_binary[y_pred_binary == 1].index
    # x_anomalies = x_test.loc[predicted_indexes]
    # y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
    # # y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
    # y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
    # y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
    # 
    # y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)
    # 
    # cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])
    # 
    # cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())
    print(f"実行結果")
    print(f"x_test {x_path}")
    print(f"y_test models/kdd99_features/y_test_df.pkl")
    print(f"第1分類モデル {path_1st}")
    print(f"第2分類モデル {path_2nd}")
    report = pd.DataFrame(classification_report(y_test.sort_index(), y_pred.sort_index(), target_names=correspondences.keys(), output_dict=True)).T.reindex(index=['normal', 'dos', 'probe', 'r2l', 'u2r', 'accuracy', 'macro avg',
       'weighted avg'])
    # print(classification_report(y_test.sort_index(), y_pred.sort_index(), target_names=correspondences.keys()))
    print(report)
    print(cm_1st)
    cm_2nd = cm_2nd.reindex(index=['true_normal', 'true_dos', 'true_probe', 'true_r2l', 'true_u2r'], columns=['normal', 'dos', 'probe', 'r2l', 'u2r'])
    print(cm_2nd)
    return cm_2nd['u2r']['true_u2r'] / cm_2nd.loc['true_u2r'].sum()

In [94]:
report = predict(data_paths[0], lgb_binary_paths[0], lgb_anomaly_paths[0])


実行結果
x_test models/kdd99_features/x_test_df.pkl
y_test models/kdd99_features/y_test_df.pkl
第1分類モデル models/lightgbm/lgb_binary_tuned_booster.model
第2分類モデル models/lightgbm/lgb_mapped_anomaly_tuned_booster.model
              precision    recall  f1-score        support
normal         0.999128  0.999595  0.999362   32102.000000
dos            0.999989  0.724658  0.840346  129181.000000
probe          0.036225  0.986716  0.069885    1355.000000
r2l            0.975275  0.954301  0.964674     372.000000
u2r            0.400000  0.588235  0.476190      17.000000
accuracy       0.781484  0.781484  0.781484       0.781484
macro avg      0.682123  0.850701  0.670091  163027.000000
weighted avg   0.991690  0.781484  0.865500  163027.000000
              normal  anomaly
true_normal    32089       13
true_anomaly      28   130897
             normal    dos  probe  r2l  u2r
true_normal   32089      1      7    4    1
true_dos          5  93612  35562    1    1
true_probe       10      0   1337    4

In [95]:
results = {}
for i in range(7):
    results[data_paths[i]] = predict(data_paths[i], lgb_binary_paths[i], lgb_anomaly_paths[i])
results

実行結果
x_test models/kdd99_features/x_test_df.pkl
y_test models/kdd99_features/y_test_df.pkl
第1分類モデル models/lightgbm/lgb_binary_tuned_booster.model
第2分類モデル models/lightgbm/lgb_mapped_anomaly_tuned_booster.model
              precision    recall  f1-score        support
normal         0.999128  0.999595  0.999362   32102.000000
dos            0.999989  0.724658  0.840346  129181.000000
probe          0.036225  0.986716  0.069885    1355.000000
r2l            0.975275  0.954301  0.964674     372.000000
u2r            0.400000  0.588235  0.476190      17.000000
accuracy       0.781484  0.781484  0.781484       0.781484
macro avg      0.682123  0.850701  0.670091  163027.000000
weighted avg   0.991690  0.781484  0.865500  163027.000000
              normal  anomaly
true_normal    32089       13
true_anomaly      28   130897
             normal    dos  probe  r2l  u2r
true_normal   32089      1      7    4    1
true_dos          5  93612  35562    1    1
true_probe       10      0   1337    4

{'models/kdd99_features/x_test_df.pkl': 0.5882352941176471,
 'models/kdd99_features/x_test-drop_25_df.pkl': 0.5882352941176471,
 'models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl': 0.7058823529411765,
 'models/kdd99_features/x_test-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl': 0.47058823529411764,
 'models/kdd99_features/x_test-drop+ae_40_df&activation=relu&epochs=5&batch_size=32.pkl': 0.5882352941176471,
 'models/kdd99_features/x_test+ae_43_df&activation=relu&epochs=5&batch_size=32.pkl': 0.5882352941176471,
 'models/kdd99_features/x_test+ae_48_df&activation=relu&epochs=5&batch_size=32.pkl': 0.0}

In [65]:
x_path = "models/kdd99_features/x_test-drop_25_df.pkl"
x_test = pd.read_pickle(x_path)
path_1st = "models/lightgbm/lgb_dropped_25_binary_tuned_booster.model"
path_2nd = "models/lightgbm/lgb_dropped+ae_30_mapped_anomaly_tuned_booster.model"
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)
y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")
model_1st: lgb.Booster = lgb.Booster(model_file=path_1st)
model_2nd: lgb.Booster = lgb.Booster(model_file=path_2nd)

# 第1段階: 正常と異常の2値分類
y_pred_binary:pd.Series = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
print(f"{y_pred_binary.value_counts()}")

y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

# 第二段階
x_path = "models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl"
x_test = pd.read_pickle(x_path)

x_anomalies: pd.DataFrame = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
# print(f"{y_pred_anomalies.value_counts()}")

y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())

swapped_correspondences = {v: k for k, v in correspondences.items()}
y_pred_value_counts = y_pred.value_counts()
y_pred_value_counts.index = y_pred_value_counts.index.map(lambda x: swapped_correspondences[x])
# print(y_pred_value_counts)

# y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
# predicted_indexes = y_pred_binary[y_pred_binary == 1].index
# x_anomalies = x_test.loc[predicted_indexes]
# y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
# # y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
# y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
# y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
# 
# y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)
# 
# cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])
# 
# cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())
print(f"実行結果：{x_path}")
print(f"第1分類モデル：{path_1st}")
print(f"第2分類モデル：{path_2nd}")
print(classification_report(y_test.sort_index(), y_pred.sort_index()))
print(cm_1st)
cm_2nd = cm_2nd.drop('normal', axis=1)

cm_2nd['u2r']['true_u2r'] / cm_2nd.loc['true_u2r'].sum()

1    130916
0     32111
dtype: int64
実行結果：models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl
第1分類モデル：models/lightgbm/lgb_dropped_25_binary_tuned_booster.model
第2分類モデル：models/lightgbm/lgb_dropped+ae_30_mapped_anomaly_tuned_booster.model
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    129181
           1       1.00      1.00      1.00     32102
           2       0.99      0.99      0.99      1355
           3       1.00      0.97      0.98       372
           4       0.62      0.59      0.61        17

    accuracy                           1.00    163027
   macro avg       0.92      0.91      0.92    163027
weighted avg       1.00      1.00      1.00    163027

              normal  anomaly
true_normal    32087       15
true_anomaly      24   130901


1.0

In [66]:
x_test.shape, y_test.shape, y_test_anomaly.shape, y_test_binary.shape

((163027, 30), (163027,), (130925,), (163027,))